In [1]:
import tensorflow as tf
import numpy as np
import time
import pickle
from IPython.display import clear_output
data=np.load('origin_data.npy')
noise_dim=data.shape[-1]
BUFFER_SIZE = 60000
BATCH_SIZE = 512
noises=np.random.normal(0,1,size=([data.shape[0], int(data.shape[-1]/2)]))
data.shape ,noises.shape

((10723, 5245), (10723, 2622))

In [2]:
class relu_constraint(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return tf.nn.relu(w)

    def get_config(self):
        return None

In [3]:
dense_dim = data.shape[-1]
def b_VAE():
    inp = tf.keras.Input((data.shape[-1],))
    noise_inp = tf.keras.Input((int(data.shape[-1]/2),))
    
    x1 = tf.keras.layers.Dense(dense_dim, name="layer11", trainable=False)(inp)
    x1 = tf.keras.layers.Dense(dense_dim, name="layer12", trainable=False)(x1)
    x1 = tf.keras.layers.Dense(dense_dim, name="layer13", trainable=False)(x1)
    x2 = tf.keras.layers.Dense(dense_dim, name="layerh1", trainable=False)(inp)
    x = x1 + x2
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.nn.relu(x)
    
    x1 = tf.keras.layers.Dense(dense_dim, name="layer21", trainable=False)(x)
    x1 = tf.keras.layers.Dense(dense_dim, name="layer22", trainable=False)(x1)
    x1 = tf.keras.layers.Dense(dense_dim/2, name="layer23", trainable=False)(x1)
    x2 = tf.keras.layers.Dense(dense_dim/2, name="layerh2", trainable=False)(x)
    x = x1 + x2
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.nn.relu(x)
    
    
    xdot = tf.keras.layers.Dense(dense_dim, name="layerd11", trainable=False)(inp)
    xdot = tf.keras.layers.Dense(dense_dim, name="layerd12", trainable=False)(xdot)
    xdot = tf.keras.layers.Dense(dense_dim/2, name="layerd13", trainable=False)(xdot)
    xdot = tf.keras.layers.Dense(dense_dim/2, name="layerd14", trainable=False)(xdot)
    xhdot = tf.keras.layers.Dense(dense_dim/2, name="layerd15", trainable=False)(inp)
    xdot = xdot + xhdot
    
    x = xdot*noise_inp + x
    xo = tf.nn.tanh(x)

    
    x1 = tf.keras.layers.Dense(dense_dim*4, kernel_regularizer=tf.keras.regularizers.l1(0.), name="layert11", trainable=False)(xo)
    x1 = tf.keras.layers.Dense(dense_dim, name="layert12", trainable=False)(x1)
    x = tf.keras.layers.Dense(dense_dim, name="layert13", trainable=False)(x1)
    x = tf.keras.layers.LayerNormalization(name="lnt1")(x)
    x = tf.nn.relu(x)
    
    x1 = tf.keras.layers.Dense(dense_dim*4, kernel_regularizer=tf.keras.regularizers.l1(0.), name="layert21", trainable=False)(x)
    x1 = tf.keras.layers.Dense(dense_dim, name="layert22", trainable=False)(x1)
    x1 = tf.keras.layers.Dense(dense_dim, name="layert23", trainable=False)(x1)
    xt = x1
    xb = tf.nn.sigmoid(xt)
    
        
    x1 = tf.keras.layers.Dense(dense_dim*2, name="layert31", kernel_regularizer=tf.keras.regularizers.l1(0.))(xb)
    x1 = tf.keras.layers.Dense(dense_dim, name="layert32")(x1)
    x2 = tf.keras.layers.Dense(dense_dim*2, name="layert33", kernel_regularizer=tf.keras.regularizers.l1(0.))(xb)
    x2 = tf.keras.layers.Dense(dense_dim, name="layert34")(x2)
    x3 = tf.keras.layers.Dense(dense_dim*2, name="layert35", kernel_regularizer=tf.keras.regularizers.l1(0.))(xb)
    x3 = tf.keras.layers.Dense(dense_dim, name="layert36")(x2)

    xv = tf.concat([x1, x2, x3, xb], 1)
    xv = tf.keras.layers.Dense(dense_dim/2, name="layert37")(xv)
    xv = tf.keras.layers.Dense(noise_dim/4, name="layert38")(xv)
    xv = tf.keras.layers.Dense(noise_dim/8, name="layert39")(xv)
    xv1 = tf.keras.layers.Dense(1, activation='sigmoid', name="layertv1")(xv)
    xv2 = tf.keras.layers.Dense(1, activation='sigmoid', name="layertv2")(xv)
    xv3 = tf.keras.layers.Dense(1, activation='sigmoid', name="layertv3")(xv)
    x = 0.33*x1*xv1 + 0.33*x2*xv2 + 0.33*x3*xv3
    
    x = 0.05*xt + 0.8*x
    x = tf.nn.sigmoid(x)
    
    return tf.keras.Model(inputs=[inp, noise_inp], outputs=x)
VAE=b_VAE()
data_test = tf.random.normal([1, data.shape[-1]])
noise = tf.random.normal([1, int(data.shape[-1]/2)])
print(VAE([data_test, noise], training=False))

tf.Tensor([[0.5110252  0.49242178 0.49941906 ... 0.48797417 0.5126568  0.51690376]], shape=(1, 5245), dtype=float32)


In [13]:
lr = 1e-1
VAE.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr),\
                 loss=tf.keras.losses.CategoricalCrossentropy())
for _ in range(5000):
    VAE.fit(x=[data,noises], y=data, batch_size=int(data.shape[0]/32)+1, epochs=20, verbose=1)

Epoch 1/20
32/32 [==============================] - 9s 293ms/step - loss: 46.9154
Epoch 2/20
32/32 [==============================] - 9s 293ms/step - loss: 46.2759
Epoch 3/20
32/32 [==============================] - 9s 293ms/step - loss: 46.1365
Epoch 4/20
32/32 [==============================] - 9s 294ms/step - loss: 46.0861
Epoch 5/20
32/32 [==============================] - 9s 294ms/step - loss: 46.0633
Epoch 6/20
32/32 [==============================] - 9s 294ms/step - loss: 46.0518
Epoch 7/20
32/32 [==============================] - 9s 295ms/step - loss: 45.9981
Epoch 8/20
32/32 [==============================] - 9s 294ms/step - loss: 46.0250
Epoch 9/20
32/32 [==============================] - 9s 295ms/step - loss: 45.9681
Epoch 10/20
32/32 [==============================] - 9s 295ms/step - loss: 46.0046
Epoch 11/20
32/32 [==============================] - 9s 294ms/step - loss: 46.0161
Epoch 12/20
 3/32 [=>............................] - ETA: 5s - loss: 47.2643

KeyboardInterrupt: 

In [9]:
noises=np.random.normal(0,1,size=([data.shape[0], int(data.shape[-1]/2)]))
avg = 0.5
erro = 0
no_c = 0
for i in range(len(data)):
    tmp  = VAE([data[i].reshape((1,data[i].shape[-1])),\
               noises[i].reshape((1,noises[i].shape[-1]))],training = False)
    tmp = tmp.numpy()[0].tolist()
    out = []
    for element in tmp:
        if(element<avg):
            out.append(0)
        else:
            out.append(1)
    c=0
    for n in (data[i] - out):
        if(n==1):
            c+=1
    if(sum(data[i]) == 0):
        tmp_c = 0
    else:
        tmp_c = c/sum(data[i])*100
    no_c += tmp_c
    clear_output(wait=True)
    print(i,"/10723")
    print("原資料未還原率:", int(tmp_c),"%")
    print("資料前10筆比對:")
    print(out[:10])
    print(data[i][:10])
    print("錯誤個數:", sum(abs(out - data[i])))
    print("錯誤比例:", sum(abs(out - data[i]))/data.shape[-1])
    erro+=sum(abs(out - data[i])) / data.shape[-1]
    #time.sleep(0.1)
clear_output(wait=True)
print("未還原平均率:", no_c/len(data), "%")
print("錯誤平均比例:", erro/len(data)*100, "%")
print("錯誤平均個數:", erro/len(data)*data.shape[-1], "個")

未還原平均率: 8.25661718279747 %
錯誤平均比例: 0.043639879602721206 %
錯誤平均個數: 2.2889116851627276 個


In [21]:
VAE.save_weights("VAE_6.h5")

In [ ]:
#存檔

In [ ]:
VAE_data = np.zeros((10723,5245))
noises=np.random.normal(0,1,size=([data.shape[0], int(data.shape[-1]/2)]))
for i in range(len(data)):
    tmp  = VAE([data[i].reshape((1,data[i].shape[-1])),\
               noises[i].reshape((1,noises[i].shape[-1]))],training = False)
    VAE_data[i] = tmp.numpy()[0]
    
np.save('VAE_data.npy',data)